In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import dmc
from process import *

Tensorflow not installed


In [3]:
evaluation_sets = ['rawMirrored', 'rawLinearSample']
data = processed_data()

In [4]:
train, test = split_data_by_id(data, 'rawMirrored')

In [5]:
len(train), len(test)

(947782, 236498)

In [6]:
ensemble = dmc.ensemble.Ensemble(train[:10000], test[:10000])

In [7]:
for k in ensemble.splits:
    print(len(ensemble.splits[k][1]), k)

7004 unknown_articleID-unknown_customerID-known_voucherID-known_productGroup
63 unknown_articleID-known_customerID-unknown_voucherID-known_productGroup
322 known_articleID-unknown_customerID-known_voucherID-known_productGroup
3 known_articleID-known_customerID-unknown_voucherID-known_productGroup
182 unknown_articleID-known_customerID-known_voucherID-known_productGroup
104 known_articleID-unknown_customerID-unknown_voucherID-known_productGroup
2311 unknown_articleID-unknown_customerID-unknown_voucherID-known_productGroup
11 known_articleID-known_customerID-known_voucherID-known_productGroup


In [8]:
import csv
import pandas as pd

file_name = 'ArticleProductgroupCustomer.csv'
with open('/Users/Fabian/Downloads/documents-export-2016-05-11/Sets_SVM_' +
                  file_name) as csv_file:
    for index, row in enumerate(csv.reader(csv_file, delimiter=',')):
        if index == 0:

            series_a = pd.Series(row[1:])
        else:
            series_b = pd.Series(row[1:])
feature_table = pd.DataFrame({'feature': series_a, "value": series_b})
unimp_fts = feature_table[feature_table['value'] == '0.0']['feature'].tolist()
print(unimp_fts)

['colorCode', 'sizeCode', 'quantity', 'price', 'rrp', 'voucherID', 'voucherAmount', 'deviceID', 'paymentMethod', 't_wsv', 't_ssv', 't_singleItemPrice', 't_singleItemPrice_diff_rrp', 't_sizeCodeNumerized', 't_unisize', 't_isTypePants', 't_isTypeBelt', 't_isTypeTop', 't_paymentWithFee_A', 't_isWeekend_A', 't_isOneSize_A', 't_customer_boughtArticleCount', 't_customer_orderCount', 't_customer_voucherCount', 't_customer_avgUnisize', 't_order_boughtArticleTypeCount', 't_order_boughtArticleCount', 't_order_duplicateCount', 't_order_sameArticlesCount', 't_order_totalPrice', 't_order_meanPrice', 't_order_cheapestPrice', 't_order_mostExpensivePrice', 't_order_totalPrice_diff_voucherAmount', 't_order_priceStd_A', 't_order_hasVoucher_A', 't_order_sameArticlesCount_DiffSize', 't_order_sameArticlesCount_DiffColor', 't_article_priceChangeSTD_A', 't_order_article_sameArticlesCount', 't_order_article_sameArticlesCount_DiffColor', 't_voucher_is10PercentVoucher', 't_voucher_is15PercentVoucher', 't_vouche

In [9]:
dropped_fts = ['t_singleItemPrice','t_isTypePants','t_isTypeBelt','t_isTypeTop'
,'t_isWeekend_A','t_isOneSize_A','t_voucher_is10PercentVoucher'
,'t_voucher_is15PercentVoucher','customerAvgUnisize', 'voucherID'] 
for drop in dropped_fts:
    unimp_fts.remove(drop)

In [10]:
ensemble.transform(binary_target=True, scalers=[dmc.transformation.normalize_raw_features]*10, 
                   ignore_features=[unimp_fts] * 10)

/Users/Fabian/Documents/HPI/Semester_6/Big_Data_Analytics_Lab/run-dmc/env/lib/python3.5/site-packages/scipy/sparse/compressed.py:730: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


/Users/Fabian/Documents/HPI/Semester_6/Big_Data_Analytics_Lab/run-dmc/env/lib/python3.5/site-packages/scipy/sparse/compressed.py:730: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


/Users/Fabian/Documents/HPI/Semester_6/Big_Data_Analytics_Lab/run-dmc/env/lib/python3.5/site-packages/scipy/sparse/compressed.py:730: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [11]:
ensemble.classify([SVM] * len(ensemble.splits))

/Users/Fabian/Documents/HPI/Semester_6/Big_Data_Analytics_Lab/run-dmc/env/lib/python3.5/site-packages/sklearn/utils/class_weight.py:62: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)


/Users/Fabian/Documents/HPI/Semester_6/Big_Data_Analytics_Lab/run-dmc/env/lib/python3.5/site-packages/sklearn/utils/class_weight.py:62: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)


/Users/Fabian/Documents/HPI/Semester_6/Big_Data_Analytics_Lab/run-dmc/env/lib/python3.5/site-packages/sklearn/utils/class_weight.py:62: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)


/Users/Fabian/Documents/HPI/Semester_6/Big_Data_Analytics_Lab/run-dmc/env/lib/python3.5/site-packages/sklearn/utils/class_weight.py:62: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)


/Users/Fabian/Documents/HPI/Semester_6/Big_Data_Analytics_Lab/run-dmc/env/lib/python3.5/site-packages/sklearn/utils/class_weight.py:62: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)


...

/Users/Fabian/Documents/HPI/Semester_6/Big_Data_Analytics_Lab/run-dmc/env/lib/python3.5/site-packages/sklearn/utils/class_weight.py:62: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)


/Users/Fabian/Documents/HPI/Semester_6/Big_Data_Analytics_Lab/run-dmc/env/lib/python3.5/site-packages/sklearn/utils/class_weight.py:62: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)


/Users/Fabian/Documents/HPI/Semester_6/Big_Data_Analytics_Lab/run-dmc/env/lib/python3.5/site-packages/sklearn/utils/class_weight.py:62: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)


/Users/Fabian/Documents/HPI/Semester_6/Big_Data_Analytics_Lab/run-dmc/env/lib/python3.5/site-packages/sklearn/utils/class_weight.py:62: DeprecationWarning: The class_weight='auto' heuristic is deprecated in 0.17 in favor of a new heuristic class_weight='balanced'. 'auto' will be removed in 0.19
  " 0.19", DeprecationWarning)


unknown_articleID-unknown_customerID-known_voucherID-known_productGroup : size  7004  prec  0.6320673900628213
--------------------------------------
unknown_articleID-known_customerID-unknown_voucherID-known_productGroup : size  63  prec  0.7142857142857143
--------------------------------------
known_articleID-unknown_customerID-known_voucherID-known_productGroup : size  322  prec  0.6863354037267081
--------------------------------------
known_articleID-known_customerID-unknown_voucherID-known_productGroup : size  3  prec  0.33333333333333337
--------------------------------------
unknown_articleID-known_customerID-known_voucherID-known_productGroup : size  182  prec  0.5274725274725275
--------------------------------------
known_articleID-unknown_customerID-unknown_voucherID-known_productGroup : size  104  prec  0.7403846153846154
--------------------------------------
unknown_articleID-unknown_customerID-unknown_voucherID-known_productGroup : size  2311  prec  0.6386845521419299


In [12]:
len(ensemble.splits)